In [1]:
from bs4 import BeautifulSoup as bs

In [2]:
import requests

In [3]:
import pprint

In [4]:
import pandas as pd

In [5]:
url = 'https://roscontrol.com/category/produkti/'

In [6]:
headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
           'Accept': '*/*'}

In [7]:
cat_selection = requests.get(url, headers=headers)

In [8]:
soup = bs(cat_selection.text, 'html.parser')

In [9]:
cat_names = soup.find_all('div',attrs={'class':'catalog__category-name'})

In [10]:
for i in range(len(cat_names)):
    cat_names[i] = cat_names[i].string

In [11]:
cat_links = soup.find_all('a',attrs={'class':'catalog__category-item'})

In [12]:
for i in range(len(cat_links)):
    cat_links[i] = cat_links[i]['href'].replace('/category/produkti/', '')

In [13]:
categories = {
    "Наименование": cat_names,
    "Ссылка": cat_links
}

categories = pd.DataFrame(categories)

In [14]:
category = categories.loc[int(input(categories["Наименование"])), "Ссылка"]

0                       Молочные продукты 
1                         Мясные продукты 
2                                   Птица 
3                     Рыба и морепродукты 
4                     Вода, соки, напитки 
5                   Хлебобулочные изделия 
6                         Детское питание 
7                                    Яйца 
8                                 Бакалея 
9                         Горячие напитки 
10                   Кондитерские изделия 
11                    Алкогольные напитки 
12                                  Соусы 
13                          Полуфабрикаты 
14                     Растительное масло 
15                               Консервы 
16                                  Снеки 
17                   Фрукты, овощи, грибы 
18                              Маргарины 
19     Готовые блюда и кулинарные изделия 
Name: Наименование, dtype: object16


In [15]:
subcat_selection = requests.get(url + category, headers=headers)

In [16]:
soup = bs(subcat_selection.text, 'html.parser')

In [17]:
subcat_names = soup.find_all('div',attrs={'class':'catalog__category-name'})

if subcat_names:
    for i in range(len(subcat_names)):
        subcat_names[i] = subcat_names[i].string

    subcat_links = soup.find_all('a',attrs={'class':'catalog__category-item'})

    for i in range(len(subcat_links)):
        subcat_links[i] = subcat_links[i]['href'].replace('/category/produkti/' + category, '')

    subcategories = {
        "Наименование": subcat_names,
        "Ссылка": subcat_links
    }

    subcategories = pd.DataFrame(subcategories)
    subcategory = subcategories.loc[int(input(subcategories["Наименование"])), "Ссылка"]
else:
    subcategory = ''

0        Чипсы 
1     Сухарики 
2      Семечки 
Name: Наименование, dtype: object2


In [18]:
response = requests.get(url + category + subcategory, headers=headers)

In [19]:
soup = bs(response.text, 'html.parser')

In [20]:
products = soup.find_all('div', attrs={'class':'wrap-product-catalog__item'})

In [21]:
products_list = []
for product in products:
    product_data = {}
    name = product.find('div', attrs={'class':'product__item-link'}).getText()
    link = 'https://roscontrol.com' + product.contents[1]['href']
    try:
        rating_status = product.find('div', attrs={'class':'rating-total-text'}).string
    except AttributeError:
        rating_status = False
    if rating_status == 'Общая оценка':
        total_rating = product.find('div', attrs={'class':'product-rating'}).contents[1].getText()
        rating_block = product.find('div', attrs={'class':'rating-block'})
        for row in rating_block.findChildren(recursive=False):
            parameter = row.findChildren(recursive=False)[0].findChildren(recursive=False)[0].getText()
            value = row.findChildren(recursive=False)[1].getText()
            product_data[parameter] = value
    elif rating_status == 'В чёрном списке':
        total_rating = 'В чёрном списке'
    elif not rating_status:
        total_rating = 'Нет информации о товаре'
    product_data['Наименование'] = name
    product_data['Ссылка'] = link
    product_data['Общий рейтинг'] = total_rating
    
    products_list.append(product_data)

In [22]:
pprint.pp(products_list, sort_dicts=False)

[{'Безопасность': '80',
  'Пищевая ценность': '16',
  'Натуральность': '85',
  'Качество': '80',
  'Наименование': 'Семечки "ВкусВилл"',
  'Ссылка': 'https://roscontrol.com/product/vkusvill-2/',
  'Общий рейтинг': ' 61 '},
 {'Безопасность': '72',
  'Пищевая ценность': '26',
  'Натуральность': '85',
  'Качество': '72',
  'Наименование': 'Семечки "Бабкины семечки"',
  'Ссылка': 'https://roscontrol.com/product/babkini-semechki/',
  'Общий рейтинг': ' 60 '},
 {'Безопасность': '75',
  'Пищевая ценность': '19',
  'Натуральность': '85',
  'Качество': '75',
  'Наименование': 'Семечки "От Мартина"',
  'Ссылка': 'https://roscontrol.com/product/semechki-ot-martina/',
  'Общий рейтинг': ' 59 '},
 {'Безопасность': '73',
  'Пищевая ценность': '18',
  'Натуральность': '85',
  'Качество': '73',
  'Наименование': 'Семечки "Золотая семечка"',
  'Ссылка': 'https://roscontrol.com/product/zolotaya-semechka/',
  'Общий рейтинг': ' 58 '},
 {'Безопасность': '65',
  'Пищевая ценность': '22',
  'Натуральность':